In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI分布式训练开始

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fget_started_with_vertex_distributed_training.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概览

本教程演示了如何使用Vertex AI Python客户端库来进行TensorFlow模型的分布式训练。

**注意：** Colab和Docker之间存在不兼容性，直到平台解决这个问题，Docker部分可能无法正常工作。

了解更多关于[Vertex AI分布式训练](https://cloud.google.com/vertex-ai/docs/training/distributed-training)的信息。

### 目标

在本教程中，您将学习如何在训练时使用`Vertex AI`的分布式训练功能。

本教程使用以下Vertex AI服务：

- Vertex AI分布式训练
- Vertex AI Reduction Server

本教程涵盖以下分布式训练技术：

- `MirroredStrategy`：在具有多个GPU的单个VM上训练。
- `MultiWorkerMirroredStrategy`（自动）：在多个VM上进行训练，并自动设置副本。
- `MultiWorkerMirroredStrategy`：在多个具有精细控制副本的VM上进行训练。
- `ReductionServer`：在多个VM上进行训练，并通过**Vertex AI Reduction Server**同步更新。
- `TPUTraining`：使用多个云TPU进行训练。

### 推荐

在谷歌云上执行端到端 MLOps 时，以下是在 Vertex AI 分布式训练中使用的最佳实践：

**单个 VM / 单个设备（OneDeviceStrategy）**

您正在进行实验，总训练数据和模型参数数量很小。

如果模型参数的数量非常小，可能无法从 GPU 中获得太多好处，可以考虑使用 VM 的 CPU。

**单个 VM / 多个计算设备（MirroredStrategy）**

模型参数数量非常大，但总训练数据很小。

**多个 VM / 多个计算设备（MultiWorkerMirroredStrategy）**

模型参数数量非常大，总训练数据也非常大。

**ReductionServer**

在大量 VM 上进行训练，模型参数更新的同步非常庞大时。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已经内置在TensorFlow中。经过训练的模型预测房屋的中位价格，单位为1K美元。

成本

本教程使用了谷歌云的可计费组件：

* Vertex AI
* Cloud Storage
* Artifact Registry

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage价格](https://cloud.google.com/storage/pricing)和[Artifact Registry价格](https://cloud.google.com/artifact-registry/pricing)，使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 重新启动运行时（仅限Colab）

为了使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。在继续下一步之前请等待完成。⚠️</b>
</div>

### 验证您在Google Colab上的笔记本环境 (仅限Colab)

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
import os

! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

在Artifact Registry中创建一个存储库

In [ ]:
# Specify the name of your repo
REPO_NAME = "vertex-distributed-unique"

# Create the repo
! gcloud artifacts repositories create {REPO_NAME} --repository-format=docker --location={LOCATION} --description="Docker repository"

# List all the repos and verify that your repo is created
! gcloud artifacts repositories list

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 初始化用于Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个具有4个Nvidia Telsa T4 GPU分配给每个VM的GPU容器映像，您需要指定：

(aiplatform.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，请指定`(None, None)`以在CPU上运行容器映像。

了解有关[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的更多信息。

**注意：**已知TF在版本2.3之前的GPU支持中在加载自定义模型时会失败。此问题已在TF版本2.3及以上版本中修复。这是由生成在服务函数中的静态图操作所导致的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建的Docker容器映像。

有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.8".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器配置

接下来，设置用于训练的机器配置。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

**注意：**以下配置不支持训练：

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

**注意：**您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## 镜像策略

当在单个虚拟机上进行训练时，您可以选择使用单个计算设备进行训练，也可以在同一虚拟机上使用多个计算设备进行训练。使用Vertex AI分布式训练，您可以指定虚拟机实例的计算设备数量和计算设备类型：CPU，GPU。

Vertex AI分布式训练支持TensorFlow模型的*tf.distribute.MirroredStrategy*。要在同一虚拟机上跨多个计算设备进行训练，您需要在Python训练脚本中执行以下附加步骤：

1. 设置*tf.distribute.MirrorStrategy*。
2. 在*tf.distribute.MirrorStrategy*范围内编译模型。这告诉MirroredStrategy应该在您的计算设备之间镜像哪些变量。
3. 将每个计算设备的批量大小增加到*num_devices* x *batch size*。

在转换期间，批处理的分发以及对模型参数的更新也是同步的。

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练作业，以及 2) 运行这个作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，具有以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，以tarball格式。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

**注意：** 不需要`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "boston-unique"

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 检查培训包

#### 包布局

在开始培训之前，查看 Python 包如何组装以用于自定义培训工作。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境的指令。

文件 `trainer/task.py` 是执行自定义培训工作的 Python 脚本。

**注意**：在引用工作池规范时，目录斜杠被替换为点（`trainer.task`），并且文件后缀（`.py`）被省略。

#### 包装

在以下单元格中，您将装配培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.8.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing cloud\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，编写训练脚本 task.py 的内容。

总结来说，task.py 脚本执行以下操作：

- 从命令行 (`--model_dir`) 或环境变量 `AIP_MODEL_DIR` 获取保存模型 artifact 的目录。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 用由 `args.epochs` 指定的 epoch 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录中（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中（`f.write(str(params))`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirrored and MultiWorker Distributed Training

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
import logging

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--batch_size', dest='batch_size',
                    default=16, type=int,
                    help='Size of a batch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

logging.info('DEVICES'  + str(device_lib.list_local_devices()))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    logging.info("Single device training")
# Single Machine, multiple compute device
elif args.distribute == 'mirrored':
    strategy = tf.distribute.MirroredStrategy()
    logging.info("Mirrored Strategy distributed training")
# Multi Machine, multiple compute device
elif args.distribute == 'multiworker':
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    logging.info("Multi-worker Strategy distributed training")
    logging.info('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
    # Single Machine, multiple TPU devices
elif args.distribute == 'tpu':
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    strategy = tf.distribute.TPUStrategy(cluster_resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

logging.info('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

def _is_chief(task_type, task_id):
    ''' Check for primary if multiworker training
    '''
    return (task_type == 'chief') or (task_type == 'worker' and task_id == 0) or task_type is None


def get_data():
    # Scaling Boston Housing data features
    def scale(feature):
        max = np.max(feature)
        feature = (feature / max).astype(float)
        return feature, max

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
        path="boston_housing.npz", test_split=0.2, seed=113
    )

    params = []
    for _ in range(13):
        x_train[_], max = scale(x_train[_])
        x_test[_], _ = scale(x_test[_])
    params.append(max)

    # store the normalization (max) value for each feature
    with tf.io.gfile.GFile(args.param_file, 'w') as f:
        f.write(str(params))
    return (x_train, y_train), (x_test, y_test)

def get_model():
    model = tf.keras.Sequential([
          tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(
          loss='mse',
          optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr)
    )
    return model

def train(model, x_train, y_train):
    NUM_WORKERS = strategy.num_replicas_in_sync
    # Here the batch size scales up by number of workers since
    # `tf.data.Dataset.batch` expects the global batch size.
    GLOBAL_BATCH_SIZE = args.batch_size * NUM_WORKERS

    model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)

    if args.distribute == 'multiworker':
        task_type, task_id = (strategy.cluster_resolver.task_type,
                              strategy.cluster_resolver.task_id)
    else:
        task_type, task_id = None, None

    if args.distribute=="tpu":
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        model.save(args.model_dir, options=save_locally)
    # single, mirrored or primary for multiworker
    elif _is_chief(task_type, task_id):
        model.save(args.model_dir)
    # non-primary workers for multi-workers
    else:
        # each worker saves their model instance to a unique temp location
        worker_dir = args.model_dir + '/workertemp_' + str(task_id)
        tf.io.gfile.makedirs(worker_dir)
        model.save(worker_dir)

with strategy.scope():
    # Creation of dataset, and model building/compiling need to be within
    # `strategy.scope()`.
    model = get_model()

(x_train, y_train), (x_test, y_test) = get_data()

train(model, x_train, y_train)

将培训脚本存储在您的云存储桶中。

接下来，将培训文件夹打包成压缩的tar文件，然后存储到您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

运行自定义的Python软件包训练任务

接下来，通过调用`run()`方法来运行自定义的任务，开始训练工作。参数与运行自定义训练任务时相同。

In [ ]:
MODEL_DIR = BUCKET_URI
MODEL_DISPLAY_NAME = "boston-unique"
CMDARGS = ["--epochs=5", "--batch_size=16", "--distribute=mirrored"]

model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    args=CMDARGS,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    base_output_dir=MODEL_DIR,
    sync=True,
)

删除自定义训练工作

在训练工作完成后，您可以使用`delete()`方法删除训练工作。在完成之前，可以使用`cancel()`方法取消训练工作。

In [ ]:
job.delete()

删除模型

同样，`delete()` 方法会删除模型。

In [ ]:
model.delete()

多工镜像策略

使用Vertex AI分布式训练，您可以使用多个VM实例进行训练。

Vertex AI分布式训练支持TensorFlow和PyTorch模型的*tf.distribute.MultiWorkerMirroredStrategy*。要在多个VM上进行训练，您需要在Python训练脚本中执行以下额外步骤：

1. 执行镜像策略的所有步骤，只不过需要将MultiWorkerMirroredStrategy替换MirroredStrategy。
2. 设置工作节点池。
3. 更改模型保存方式，使非主要工作节点将其模型实例保存到唯一的临时目录中。

**注意:** 您无需构造TF_CONFIG环境变量，Vertex AI分布式训练会自动构造它。

了解更多关于[分布式训练](https://cloud.google.com/vertex-ai/docs/training/distributed-training)的信息。

### Worker pools

如果您在Vertex AI中运行分布式训练作业，则需要在训练集群中指定多台机器（节点）。训练服务会为您指定的机器类型分配资源。在给定节点上运行的作业称为副本。具有相同配置的一组副本称为工作池。

训练集群中的每个副本都被指定为分布式训练中的一个单一角色或任务。例如：

- **主副本**：指定一个副本作为主副本。此任务负责管理其他副本并报告整个作业的状态。

- **工作者**：一个或多个副本可以被指定为工作者。这些副本按照您在作业配置中指定的方式执行其工作。

- **参数服务器**：如果您的ML框架支持，可以指定一个或多个副本作为参数服务器。这些副本存储模型参数并在工作者之间协调共享模型状态。

- **评估器**：如果您的ML框架支持，可以指定一个或多个副本为评估器。这些副本可用于评估您的模型。如果您正在使用TensorFlow，请注意，TensorFlow通常希望您最多使用一个评估器。

要配置分布式训练作业，请定义您的工作池列表（workerPoolSpecs[]），为每种任务指定一个WorkerPoolSpec：

**注意：**工作池是按顺序排列的（0..3）：

**workerPoolSpecs[0]**：主、首席、调度程序或“主”

**workerPoolSpecs[1]**：次要、副本、工作者

**workerPoolSpecs[2]**：参数服务器、减少服务器

**workerPoolSpecs[3]**：评估器

### Multi-Worker Mirrored Strategy的分布式训练选项

设置工作池取决于您用于训练的Vertex AI方法。

**CustomTrainingJob** / **CustomContainerTrainingJob** / **CustomPythonPackageTrainingJob**

`replica_count`包括主要和次要（replica_count-1），并且共享相同的机器类型和加速器。

您无法指定参数服务器或评估节点。

**CustomJob**

您可以指定一个`worker_pool_spec`，在其中可以为四个工作池中的每一个指定详细设置。

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要进行两个步骤：1）创建一个自定义训练任务，以及2）运行该任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要提供以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `python_package_gcs_uri`：Python训练包的位置（.tarball文件）。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

**注意：** 没有`requirements`参数。您需要在Python包中的`setup.py`脚本中指定任何依赖关系。

In [ ]:
DISPLAY_NAME = "boston-unique"

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_boston.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

运行自定义的Python包训练作业

接下来，通过调用`run()`方法来运行自定义作业，开始训练作业。参数与运行CustomTrainingJob时相同。

In [ ]:
MODEL_DIR = BUCKET_URI

CMDARGS = ["--epochs=5", "--batch_size=16", "--distribute=multiworker"]

try:
    model = job.run(
        model_display_name=MODEL_DISPLAY_NAME,
        args=CMDARGS,
        replica_count=4,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
except Exception as e:
    # may fail duing model.save() -- seems to be some issue when merging checkpoints from the workers
    print(e)

### 删除自定义训练作业

在训练作业完成后，您可以使用 `delete()` 方法删除训练作业。在完成之前，可以使用 `cancel()` 方法取消训练作业。

In [ ]:
job.delete()

### 使用CustomJob进行分布式多工作器训练

使用`CustomJob`进行分布式多工作器训练具有对主副本的细粒度控制以及可选择为参数服务器和评估器指定工作器池的优势。创建一个`CustomJob`包括以下步骤：

1. 为每个工作器池指定详细信息。
2. 将训练包嵌入Docker镜像中。

### 构建一个自定义的训练容器

要使用您自己的自定义训练容器，您需要构建一个 Docker 文件，并将您的训练脚本嵌入到容器中。

#### 编写Docker文件内容

容器化您的代码的第一步是创建一个Docker文件。在Docker文件中，包括运行容器镜像所需的所有命令。Docker安装您指定的所有库，并设置训练代码的入口点。

Docker文件中的步骤包括:

1. 从 TensorFlow 仓库中安装预定义的深度学习镜像。
2. 将 Python 训练代码复制到镜像中，随后会展示。
3. 将 Python 训练脚本的入口点设置为 `trainer/task.py`。请注意，ENTRYPOINT 命令中省略了 `.py`，因为这是隐含的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

#### 配置身份验证到您的仓库

在推送或拉取容器镜像之前，配置Docker使用gcloud命令行工具对您的地区的Artifact Registry进行请求的身份验证。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB:
    ! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

#### 本地构建容器镜像

使用 `docker build` 命令本地构建您的容器镜像。

使用 `-t` 标志提供仓库名称作为标签。了解更多关于将镜像添加到[Artifact Registry](https://cloud.google.com/artifact-registry/docs/docker/store-docker-container-images#add-image)。

In [ ]:
# Specify the repository name where the image needs to be uploaded
TRAIN_IMAGE = f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/boston:v1"

In [ ]:
if not IS_COLAB:
    # build the image locally with the repository name as a tag
    ! docker build custom -t $TRAIN_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

# 在本地测试容器

在笔记本实例中运行容器5个周期，以确保它可以正常工作。

**注意**：如果您在笔记本环境中运行此教程，例如 Vertex AI workbench，请继续以下步骤。而如果您在 Colab 上运行，请跳转到"[在Colab中运行](#在Colab中运行)"部分。

In [ ]:
if not IS_COLAB:
    ! docker run $TRAIN_IMAGE --epochs=5 --model-dir=./

#### 注册自定义容器

在本地运行容器完成后，将其推送到Artifact Registry。

在配置身份验证和为本地映像打标签之后，您可以将映像推送到您创建的存储库中。

要推送Docker映像，请运行以下命令：

In [ ]:
if not IS_COLAB:
    ! docker push $TRAIN_IMAGE

#### 在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $TRAIN_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build custom -t $2
docker run $2 --epochs=5 --model-dir=./
docker push $2
kill $(jobs -p)

主要工作池

主要工作池（索引0）协调所有其他副本完成的工作。 将“replica_count”设置为1。

由于此工作者是协调而不是训练，因此请使用通用目的的 CPU，而不是 GPU。

了解有关[用于训练的机器类型](https://cloud.google.com/vertex-ai/docs/training/configure-compute#machine-types)的更多信息。

In [ ]:
PRIMARY_COMPUTE = "n2-highcpu-64"

MODEL_DIR = BUCKET_URI

CMDARGS = [
    "--model-dir=" + MODEL_DIR,
    "--epochs=5",
    "--batch_size=16",
    "--distribute=multiworker",
]

CONTAINER_SPEC = {"image_uri": TRAIN_IMAGE, "command": "trainer.task", "args": CMDARGS}

PRIMARY_WORKER_POOL = {
    "replica_count": 1,
    "machine_spec": {"machine_type": PRIMARY_COMPUTE, "accelerator_count": 0},
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS = [PRIMARY_WORKER_POOL]

培训工人池

次要工人池（索引1）执行模型培训。每个副本都安装有您的培训软件包的实例。

每个副本可能具有一个（单设备培训）或多个（镜像）用于培训的计算设备。

In [ ]:
TRAIN_WORKER_POOL = {
    "replica_count": 4,
    "machine_spec": {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_count": TRAIN_NGPU,
        "accelerator_type": TRAIN_GPU,
    },
    "container_spec": CONTAINER_SPEC,
}

WORKER_POOL_SPECS.append(TRAIN_WORKER_POOL)

使用工作人员池规范创建CustomJob

接下来，为多工作人员分布式培训工作创建CustomJob。

指定以下参数：

- `display_name`：自定义工作的显示名称。

- `worker_pool_specs`：每个工作人员池的详细规格。

In [ ]:
DISPLAY_NAME = "boston-unique"

job = aiplatform.CustomJob(
    display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS
)

运行定制作业

接下来，运行定制作业。

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

### 删除自定义训练任务

训练任务完成后，您可以使用 `delete()` 方法删除训练任务。在完成之前，训练任务可以使用 `cancel()` 方法取消。

In [ ]:
job.delete()

## 降幅服务器

为了加快大型模型的训练速度，许多工程团队正在采用分布式训练，使用ML加速器的规模扩展集群。然而，在规模化的分布式训练中会带来一系列挑战。特别是节点之间有限的网络带宽使得分布式训练的性能优化困难，特别是对于大型集群配置。

Vertex AI降幅服务器优化了NVIDIA GPU上同步数据并行算法的多节点分布式训练的带宽和延迟。同步数据并行是许多广泛采用的分布式训练框架的基础，包括TensorFlow的MultiWorkerMirroredStrategy、Horovod和PyTorch Distributed。通过优化这些框架使用的all-reduce集合操作的带宽使用和延迟，降幅服务器可以降低大型训练作业的时间和成本。

了解更多关于[在Vertex AI上使用降幅服务器优化训练性能](https://cloud.google.com/blog/topics/developers-practitioners/optimize-training-performance-reduction-server-vertex-ai)。

In [ ]:
reduction_server_count = 1
reduction_server_machine_type = "n1-highcpu-16"
reduction_server_image_uri = (
    "us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest"
)

PARAMETER_POOL = {
    "replica_count": reduction_server_count,
    "machine_spec": {
        "machine_type": reduction_server_machine_type,
    },
    "container_spec": {"image_uri": reduction_server_image_uri},
}
WORKER_POOL_SPECS.append(PARAMETER_POOL)

### 使用工作池规范创建CustomJob

接下来，为多工作人员分布式培训工作创建`CustomJob`。

指定以下参数：

- `display_name`：自定义作业的显示名称。

- `worker_pool_specs`：每个工作池的详细规范。

In [ ]:
DISPLAY_NAME = "boston-unique"

job = aiplatform.CustomJob(
    display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS
)

运行自定义作业

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

### 删除自定义训练作业

在训练作业完成后，您可以使用 `delete()` 方法来删除训练作业。在完成之前，训练作业可以使用 `cancel()` 方法取消。

In [ ]:
job.delete()

## 云 TPU 训练

为了进一步加快训练速度，您的组织可以利用谷歌的云张量处理单元 (TPU) 云。

云 TPU 是谷歌产品如翻译、照片、搜索、助手和 Gmail 所使用的定制机器学习 ASIC。云 TPU 旨在在谷歌云上运行尖端机器学习模型与 AI 服务。它的定制高速网络在单个云中提供超过 100 petaflops 的性能。

了解有关[云 TPU](https://cloud.google.com/tpu)的更多信息。

**注意**：TPU 虚拟机训练目前是一项可选择的功能。您的 GCP 项目必须首先被添加到功能允许列表中。请通过电子邮件将您的项目信息 (项目 ID/编号) 发送至 vertex-ai-tpu-vm-training-support@google.com 以允许列表。您的项目准备好后将收到一封电子邮件。

### 为 TPU 训练编写 Docker 文件

目前，还没有专门用于在 TPU 上进行训练的 Vertex AI Docker 镜像。但是，您可以自己制作一个，按照以下步骤操作：

1. 创建一个纯净的 Python 3 镜像（例如，`python3:8`）。
2. 获取并安装 TPU 库（`libtpu.so`）。
3. 从您的训练包中复制训练器代码。

In [ ]:
%%writefile custom/Dockerfile
FROM python:3.8

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip3 install tensorflow-datasets

# Install TPU Tensorflow and dependencies.
# libtpu.so must be under the '/lib' directory.
RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/20210525/libtpu.so -O /lib/libtpu.so
RUN chmod 777 /lib/libtpu.so

RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/20210525/tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN pip3 install tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN rm tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

构建并推送Docker镜像到Artifact Registry。

In [ ]:
# Specify the repository name where the image needs to be uploaded
TPU_TRAIN_IMAGE = f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/tpu-train:latest"

if not IS_COLAB:
    # build the image locally with the repository name as a tag
    ! docker build --quiet -t $TPU_TRAIN_IMAGE custom
    # push the image to artifact registry
    ! docker push $TPU_TRAIN_IMAGE

如果您正在使用Colab，请运行下面的单元格来构建并推送图像。

In [ ]:
%%bash -s $IS_COLAB $TPU_TRAIN_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build --quiet custom -t $2
docker push $2
kill $(jobs -p)

TPU工作人员规范池

接下来，创建工作人员规范池。

对于TPUs：

- 仅创建一个工作人员池（主要）。
- 将机器类型设置为`cloud-tpu`。
- 将加速器类型设置为`TPU`。

In [ ]:
# Use TPU Accelerators. Temporarily using numeric codes, until types are added to the SDK
#   6 = TPU_V2
#   7 = TPU_V3
TRAIN_TPU, TRAIN_NTPU = (7, 8)
TRAIN_COMPUTE = "cloud-tpu"


if not TRAIN_NTPU or TRAIN_NTPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "tpu"
print(TRAIN_STRATEGY)

EPOCHS = 20
STEPS = 10000

TRAINER_ARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]


WORKER_POOL_SPECS = [
    {
        "container_spec": {
            "args": TRAINER_ARGS,
            "image_uri": TPU_TRAIN_IMAGE,
        },
        "replica_count": 1,
        "machine_spec": {
            "machine_type": TRAIN_COMPUTE,
            "accelerator_type": TRAIN_TPU,
            "accelerator_count": TRAIN_NTPU,
        },
    }
]

print(WORKER_POOL_SPECS[0])

### 使用worker pool规格创建CustomJob

接下来，为多工作节点分布式训练作业创建一个`CustomJob`：

- `display_name`：自定义作业的显示名称。

- `worker_pool_specs`：每个工作节点池的详细规格。

In [ ]:
DISPLAY_NAME = "boston-unique"

job = aiplatform.CustomJob(
    display_name=DISPLAY_NAME, worker_pool_specs=WORKER_POOL_SPECS
)

### 运行自定义作业

接下来，运行自定义作业。

In [ ]:
try:
    job.run(sync=True)
except Exception as e:
    # may fail in multi-worker to find startup script
    print(e)

### 删除自定义训练作业

在训练作业完成后，您可以使用 `delete()` 方法删除训练作业。在完成之前，可以使用 `cancel()` 方法取消训练作业。

In [ ]:
job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：


- 云存储存储桶
- 工件存储库

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete the repo in Artifact Registry
!gcloud artifacts repositories delete --location=$LOCATION $REPO_NAME --quiet

# Delete the locally generated files
!rm -rf custom/
!rm custom.tar.gz